<a href="https://colab.research.google.com/github/SteBaum/Nigeria-Bank/blob/main/Yolov5_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Connect to Google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
# Install all required modules
!pip install torch==1.5.1+cu101 torchvision==0.6.1+cu101 -f https://download.pytorch.org/whl/torch_stable.html
!pip install numpy==1.19.5
!pip install PyYAML==5.3.1
!pip install git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 14.8 MB 5.1 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.17.0
    Uninstalling numpy-1.17.0:
      Successfully uninstalled numpy-1.17.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.11.0 requires torch==1.10.0, but you have torch 1.5.1+cu101 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-pc32fj66
  Running command git clone -q https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-pc32fj66


In [3]:
# Optain libraries for pytorch
!git clone https://github.com/NVIDIA/apex && cd apex && pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" . --user && cd .. && rm -rf apex

Cloning into 'apex'...
remote: Enumerating objects: 8879, done.
remote: Counting objects: 100% (112/112), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 8879 (delta 64), reused 77 (delta 46), pack-reused 8767
Receiving objects: 100% (8879/8879), 14.52 MiB | 28.21 MiB/s, done.
Resolving deltas: 100% (6045/6045), done.
/usr/local/lib/python3.7/dist-packages/pip/_internal/commands/install.py:232: UserWarning: Disabling all use of wheels due to the use of --build-option / --global-option / --install-option.
  cmdoptions.check_install_build_global(options)
Using pip 21.1.3 from /usr/local/lib/python3.7/dist-packages/pip (python 3.7)
User install by explicit request
Created temporary directory: /tmp/pip-ephem-wheel-cache-ab6n_vtk
Created temporary directory: /tmp/pip-req-tracker-glvk9s6q
Initialized build tracking at /tmp/pip-req-tracker-glvk9s6q
Created build tracker: /tmp/pip-req-tracker-glvk9s6q
Entered build tracker: /tmp/pip-req-tracker-glvk9s6q
Created temporary d

In [2]:
# Import modules

from pathlib import Path
from tqdm import tqdm
import numpy as np
import json
import urllib
import PIL.Image as Image
import cv2
import torch
import torchvision
from IPython.display import display
from sklearn.model_selection import train_test_split
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
import pandas as pd
import json
import ast
import os
import re

In [62]:
# Load csv dataframe
bankreports = pd.read_csv("/content/gdrive/MyDrive/Liberia_bank/via_project_7Jan2022_14h36m_csv.csv")

In [63]:
bankreports

,filename,file_size,file_attributes,region_count,region_id,region_shape_attributes,region_attributes
0,1-2.jpg,144789,{},2,0,"{""name"":""rect"",""x"":20,""y"":136,""width"":200,""hei...",{}
1,1-2.jpg,144789,{},2,1,"{""name"":""rect"",""x"":276,""y"":39,""width"":201,""hei...",{}
2,1-2-pages.jpg,120719,{},2,0,"{""name"":""rect"",""x"":14,""y"":200,""width"":220,""hei...",{}
3,1-2-pages.jpg,120719,{},2,1,"{""name"":""rect"",""x"":15,""y"":107,""width"":219,""hei...",{}
4,Addiko-bank-st.jpg,187754,{},2,0,"{""name"":""rect"",""x"":36,""y"":377,""width"":393,""hei...",{}
...,...,...,...,...,...,...,...
1586,Yemen-Gulf-bank-statement-template-Word-and-PD...,147935,{},2,1,"{""name"":""rect"",""x"":49,""y"":359,""width"":373,""hei...",{}
1587,Yemen-Gulf-Bank-statement-template-in-Excel-an...,172033,{},2,0,"{""name"":""rect"",""x"":31,""y"":274,""width"":405,""hei...",{}
1588,Yemen-Gulf-Bank-statement-template-in-Excel-an...,172033,{},2,1,"{""name"":""rect"",""x"":31,""y"":405,""width"":404,""hei...",{}
1589,Yemen-International-bank-of-Yemen-proof-of-add...,208654,{},1,0,"{""name"":""rect"",""x"":52,""y"":222,""width"":370,""hei...",{}


In [64]:
# Check the region_shape_attributes column
bankreports.region_shape_attributes[1]

'{"name":"rect","x":276,"y":39,"width":201,"height":295}'

In [65]:
# Seperate the region_shape_attributes into different columns
df = pd.json_normalize(bankreports.region_shape_attributes.apply(ast.literal_eval))

In [66]:
# Join the newly created columns to the dataframe
bankreports = bankreports.join(df)

In [67]:
bankreports.head()

,filename,file_size,file_attributes,region_count,region_id,region_shape_attributes,region_attributes,name,x,y,width,height
0,1-2.jpg,144789,{},2,0,"{""name"":""rect"",""x"":20,""y"":136,""width"":200,""hei...",{},rect,20,136,200,200
1,1-2.jpg,144789,{},2,1,"{""name"":""rect"",""x"":276,""y"":39,""width"":201,""hei...",{},rect,276,39,201,295
2,1-2-pages.jpg,120719,{},2,0,"{""name"":""rect"",""x"":14,""y"":200,""width"":220,""hei...",{},rect,14,200,220,121
3,1-2-pages.jpg,120719,{},2,1,"{""name"":""rect"",""x"":15,""y"":107,""width"":219,""hei...",{},rect,15,107,219,27
4,Addiko-bank-st.jpg,187754,{},2,0,"{""name"":""rect"",""x"":36,""y"":377,""width"":393,""hei...",{},rect,36,377,393,128


In [68]:
type(bankreports.filename[27])

str

In [70]:
# Split the image files into train and validation samples, training sample being 90%
train_reports, val_reports = train_test_split(list(bankreports['filename'].drop_duplicates()), test_size=0.1)
len(train_reports), len(val_reports)

(846, 94)

In [72]:
# Define a function that creaets the dataset with
# the following images and text annotation files.

def create_dataset(reports, dataset_type):
  images_path = Path(f"content/gdrive/MyDrive/Liberia_bank/reports/images/{dataset_type}")
  images_path.mkdir(parents=True, exist_ok=True) # path and folder for image file

  labels_path = Path(f"content/gdrive/MyDrive/Liberia_bank/reports/labels/{dataset_type}")
  labels_path.mkdir(parents=True, exist_ok=True) # path and folder for annoation files

  for item in reports:
    #iterate over rows in the dataframe
    #try:    
      abs_path = os.path.join("/content/gdrive/MyDrive/Liberia_bank/bankreport_images", item) #create path
            
  # Open the image files and save them in the corresponding folders
      img = Image.open(abs_path)
      img = img.convert("RGB")
      img.save(str(images_path / item), "JPEG")
      label_name = item.replace('.jpg',".txt" )
      with (labels_path / label_name).open("w") as label_file:
        for idx, col in bankreports.iterrows():
          if item == col[0]:
         # Open a text file with the annotation coordinates
            x = bankreports.iloc[idx]["x"]
            y = bankreports.iloc[idx]["y"]
            width = bankreports.iloc[idx]["width"]
            height = bankreports.iloc[idx]["height"]

          # Write the annotation coordinates into a text file
            label_file.write(f"0 {x + width / 2} {y + height / 2} {width} {height}\n")
            
    #except: # accept the error and continue
      #continue


In [73]:
create_dataset(train_reports, 'train')
create_dataset(val_reports, 'val')

In [74]:
# Get some specifications for Yolo
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!git checkout ec72eea62bf5bb86b0272f2e65e413957533507f

Cloning into 'yolov5'...
remote: Enumerating objects: 10649, done.
remote: Total 10649 (delta 0), reused 0 (delta 0), pack-reused 10649
Receiving objects: 100% (10649/10649), 10.85 MiB | 32.59 MiB/s, done.
Resolving deltas: 100% (7354/7354), done.
/content/yolov5
Note: checking out 'ec72eea62bf5bb86b0272f2e65e413957533507f'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at ec72eea Merge remote-tracking branch 'origin/master'


In [75]:
# Configurating Yolo
!gdown --id 1ZycPS5Ft_0vlfgHnLsfvZPhcH6qOAqBO -O data/clothing.yaml
!gdown --id 1czESPsKbOWZF7_PkCcvRfTiUUJfpx12i -O models/yolov5x.yaml

Downloading...
From: https://drive.google.com/uc?id=1ZycPS5Ft_0vlfgHnLsfvZPhcH6qOAqBO
To: /content/yolov5/data/clothing.yaml
100% 172/172 [00:00<00:00, 80.3kB/s]
Downloading...
From: https://drive.google.com/uc?id=1czESPsKbOWZF7_PkCcvRfTiUUJfpx12i
To: /content/yolov5/models/yolov5x.yaml
100% 1.58k/1.58k [00:00<00:00, 1.39MB/s]


In [77]:
!python train.py --img 640 --batch 4 --epochs 30 \
  --data ./data/clothing.yaml --cfg ./models/yolov5x.yaml --weights yolov5x.pt \
  --name yolov5x_clothing --cache

Apex recommended for faster mixed precision training: https://github.com/NVIDIA/apex
{'lr0': 0.01, 'momentum': 0.937, 'weight_decay': 0.0005, 'giou': 0.05, 'cls': 0.58, 'cls_pw': 1.0, 'obj': 1.0, 'obj_pw': 1.0, 'iou_t': 0.2, 'anchor_t': 4.0, 'fl_gamma': 0.0, 'hsv_h': 0.014, 'hsv_s': 0.68, 'hsv_v': 0.36, 'degrees': 0.0, 'translate': 0.0, 'scale': 0.5, 'shear': 0.0}
Namespace(adam=False, batch_size=4, bucket='', cache_images=True, cfg='./models/yolov5x.yaml', data='./data/clothing.yaml', device='', epochs=30, evolve=False, img_size=[640], multi_scale=False, name='yolov5x_clothing', noautoanchor=False, nosave=False, notest=False, rect=False, resume=False, single_cls=False, weights='yolov5x.pt')
Using CUDA device0 _CudaDeviceProperties(name='Tesla P100-PCIE-16GB', total_memory=16280MB)

Start Tensorboard with "tensorboard --logdir=runs", view at http://localhost:6006/

              from  n    params  module                                  arguments                     
  0             -1